## Análisis de métodos de detección y traducción de licitaciones

#### input: histórico de licitaciones
#### output: traducción de los atributos relevantes del histórico de licitaciones

### Autor: Silvia García, Data Scientist, Grupo EVM

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors
import gensim

import warnings
warnings.filterwarnings("ignore")

Información relevante de tensorflow, cuda, cudnn, pytorch y transformers

- https://towardsdatascience.com/setting-up-tensorflow-gpu-with-cuda-and-anaconda-onwindows-2ee9c39b5c44
- https://www.tensorflow.org/install/source#gpu
- https://pytorch.org/get-started/locally/

- https://github.com/detectlanguage/detectlanguage-python
- https://detectlanguage.com/private

### 1. Lectura del df

In [2]:
# Reading the data
df = pd.read_csv("main_licitaciones_bbdd.csv", error_bad_lines = False)

In [3]:
print('el número de licitaciones es de', len(df))
df.head(2)

el número de licitaciones es de 849412


,Identificador,Número de expediente,Link licitación,Objeto del Contrato,Estado,Vigente/Anulada/Archivada,Tipo de procedimiento,Tramitación,Forma de presentación de la oferta,Órgano de Contratación,...,Número de ofertas recibidas por licitación/lote,Resultado licitación/lote,Identificador Adjudicatario de la licitación/lote,Importe adjudicación sin impuestos licitación/lote,Primera publicación,Fecha actualización,Código Lugar de ejecución,Tipo Licitación,idioma,translate
0,8689190,300/2021/00220,https://contrataciondelestado.es/wps/poc?uri=d...,"Escuela de deportes alternativos, para el fome...",Resuelta,VIGENTE,Abierto simplificado,Ordinaria,Electrónica,Distrito de Latina,...,2,Adjudicado,G82270489,"20000,0",2021-11-15,2021-12-30,ES300,Licitación Directa,Language.SPANISH,"Escuela de deportes alternativos, para el fome..."
1,8723412,2021/066 PASS,https://contrataciondelestado.es/wps/poc?uri=d...,Suministro (Suscripción) de licencias de softw...,Adjudicada,VIGENTE,Abierto simplificado,Ordinaria,Electrónica,Presidencia de la Autoridad Independiente de R...,...,6,Adjudicado,B84852391,"17656,32",2021-11-18,2021-12-30,ES300,Licitación Directa,Language.SPANISH,Suministro (Suscripción) de licencias de softw...


In [4]:
df.columns

Index(['Identificador', 'Número de expediente', 'Link licitación',
       'Objeto del Contrato', 'Estado', 'Vigente/Anulada/Archivada',
       'Tipo de procedimiento', 'Tramitación',
       'Forma de presentación de la oferta', 'Órgano de Contratación',
       'Tipo de contrato', 'Lote', 'CPV', 'Sistema de contratación',
       'Directiva de aplicación', 'Subcontratación permitida',
       'Valor estimado del contrato', 'Presupuesto base sin impuestos',
       'Con Adjudicatario', 'Número de ofertas recibidas por licitación/lote',
       'Resultado licitación/lote',
       'Identificador Adjudicatario de la licitación/lote',
       'Importe adjudicación sin impuestos licitación/lote',
       'Primera publicación', 'Fecha actualización',
       'Código Lugar de ejecución', 'Tipo Licitación', 'idioma', 'translate'],
      dtype='object')

### 2. Detección de idioma de la licitación

In [5]:
#En el plan gratis hay una limitación de 1000 traducciones al día. No me sirve para el proyecto
#https://detectlanguage.com/private/plan_changes/new
#import detectlanguage

#detectlanguage.configuration.api_key = "fcf87902b3e0e5516758bb7278ee87e7"
#detectlanguage.detect('hola')

In [6]:
#https://github.com/pemistahl/lingua-py

from lingua import Language, LanguageDetectorBuilder

languages = [Language.CATALAN, Language.SPANISH]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
detector.detect_language_of("present contracte és atorgar l\'ús privatiu del")

Language.CATALAN

In [15]:
# En la segunda vuelta, detectamos el idioma de la columna ya traducida
df['idioma_translate'] = df['translate'].apply(lambda x: detector.detect_language_of(x))

In [19]:
df.groupby('idioma').count()

,Identificador,Número de expediente,Link licitación,Objeto del Contrato,Estado,Vigente/Anulada/Archivada,Tipo de procedimiento,Tramitación,Forma de presentación de la oferta,Órgano de Contratación,...,Número de ofertas recibidas por licitación/lote,Resultado licitación/lote,Identificador Adjudicatario de la licitación/lote,Importe adjudicación sin impuestos licitación/lote,Primera publicación,Fecha actualización,Código Lugar de ejecución,Tipo Licitación,translate,idioma_translate
idioma,,,,,,,,,,,,,,,,,,,,,
Language.CATALAN,143910,143910,143910,143910,143910,143910,143910,143910,143910,143910,...,143910,143910,143910,143910,143910,143910,143910,143910,143910,143909
Language.SPANISH,705173,705173,705173,705173,705173,705173,705173,705173,705173,705173,...,705173,705173,705173,705173,705173,705173,705173,705173,705173,705173


In [16]:
# Después de la traducción quedan unas 18k licitaciones sin traducir bien
df.groupby('idioma_translate').count()

,Identificador,Número de expediente,Link licitación,Objeto del Contrato,Estado,Vigente/Anulada/Archivada,Tipo de procedimiento,Tramitación,Forma de presentación de la oferta,Órgano de Contratación,...,Número de ofertas recibidas por licitación/lote,Resultado licitación/lote,Identificador Adjudicatario de la licitación/lote,Importe adjudicación sin impuestos licitación/lote,Primera publicación,Fecha actualización,Código Lugar de ejecución,Tipo Licitación,idioma,translate
idioma_translate,,,,,,,,,,,,,,,,,,,,,
Language.CATALAN,17992,17992,17992,17992,17992,17992,17992,17992,17992,17992,...,17992,17992,17992,17992,17992,17992,17992,17992,17992,17992
Language.SPANISH,831090,831090,831090,831090,831090,831090,831090,831090,831090,831090,...,831090,831090,831090,831090,831090,831090,831090,831090,831090,831090


In [36]:
#Guardado
df.to_csv('main_licitaciones_bbdd_idioma.csv', index = False)

In [8]:
df.loc[618520, 'Objeto del Contrato']

'L\'objecte del present contracte és atorgar l\'ús privatiu del domini públic de l\'espai destinat a bar restaurant situat a l\'e difici de l\'equipament cultural de la societat recreativa "La Unió Fraternal" ubicat a la Riera Ginjolers. Aquest edifici és un equipament cultural de l\'Ajuntament de Roses, que es pretén configurar\ncom un espai per al desenvolupament d\'activitats educatives, culturals i d\'oci, essent el bar un complement dels diversos serveis del conjunt de l\'equipament, i en conseqüència, ha d\'oferir una imatge singular, atractiva i respectuosa per a tots els sectors de la població, col·laborant en el desenvolupament d\'a ctivitats que s\'hi promocionin. Per tal de garantir una correcta coordinació entre els usos de les diferents dependències de l\'equipament, l\'Ajuntament establirà els mecanismes perquè es pugui realitzar amb efectivitat aquesta coordinació. S\'adjunta plànol descriptiu de la zona a ocupar dins l\'edifici com a Annex III.'

### 3. Traducción automática

Utilizamos modelos de Hugging Face ya entrenados para hacer la traducción de catalán a castellano.

El modelo es: https://huggingface.co/Helsinki-NLP/opus-mt-ca-en

In [5]:
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MarianMTModel, MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ca-es")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ca-es")

In [10]:
def translate_row(x):
    #inputs:
    #x: string to trasnlate from catalan to spanish
    
    batch = tokenizer([x], return_tensors = "pt")
    generated_ids = model.generate(**batch)
    trans = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)[0]
    
    return trans

In [12]:
%%time
df['translate'] = df.apply(lambda x: translate_row(x['Objeto del Contrato']) if ((x['idioma'] == 'Language.CATALAN') 
                                                                                & (len(x['Objeto del Contrato']) <= 512))
                           else x['Objeto del Contrato'], axis = 1)

CPU times: total: 13d 17h 24min 4s
Wall time: 2d 7h 35min 41s


In [14]:
#Guardado
df.to_csv('main_licitaciones_bbdd_idioma_traduccion.csv', index = False)

In [33]:
df_.loc[608, 'Objeto del Contrato']

"Nova xarxa d'aigua potable, aigua pluvial i adaptació a la legislacio en matèria d'accessibilitat"

In [34]:
df_.loc[608, 'translate']

'Nueva red de agua potable, agua pluvial y adaptación a la legislacion en materia de accesibilidad'